# Chapter 8: Applying Machine Learning to Sentiment Analysis

In [1]:
import pyprind
import pandas as pd
import os

KeyboardInterrupt: 

In [ ]:
pbar = pyprind.ProgBar(50000)
labels = {'pos': 1, 'neg' : 0}
df = pd.DataFrame()

In [ ]:
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/{}/{}'.format(s, l)
        for file in os.listdir(path):
            with open(os.path.join(path,file), 
                      'r', encoding='utf8') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()

In [ ]:
import numpy as np

In [ ]:
np.random.seed(0)

In [ ]:
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.to_csv('movie_data.csv', index=False, encoding='utf8')

In [ ]:
df = pd.read_csv('movie_data.csv')
df.head(3)

## Bag of Words Model

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer

In [ ]:
count = CountVectorizer()

In [ ]:
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining and the weather is sweet'])

In [ ]:
bag = count.fit_transform(docs)

In [ ]:
print(count.vocabulary_)

In [ ]:
print(bag.toarray())

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf = TfidfTransformer()

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

### Cleaning Text Data

In [ ]:
df.columns = ['review', 'sentiment']

In [ ]:
df.loc[0, 'review'][-50:]

In [ ]:
import re

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-','')
    return text

In [ ]:
preprocessor('<a> This :) is a :( test </a>)')

In [ ]:
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
tokenizer('runners like running and thus they run')

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
porter = PorterStemmer()

In [ ]:
def tokenizer_porter(text):
    return [ porter.stem(word) for
           word in text.split()]

In [ ]:
tokenizer_porter('runners like running and thus they run')

#### Remove Stop Words

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('english')

In [ ]:
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

## Training a logistic regression model for document classification

In [ ]:
X_train = df.loc[:25000, 'review'].values
X_test = df.loc[25000:, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

In [ ]:
param_grid = [{'vect__ngram_range':[(1,1)],
              'vect__stop_words':[stop,None],
              'vect__tokenizer':[tokenizer, tokenizer_porter],
              'clf__penalty':['l1', 'l2'],
              'clf__C':[1.0, 10.0, 100.0]},
              {'vect__ngram_range':[(1,1)],
              'vect__stop_words':[stop,None],
              'vect__tokenizer':[tokenizer, tokenizer_porter],
              'vect__use_idf':[False],
              'vect__norm':[None],
              'clf__penalty': ['l1', 'l2'],
              'clf__C':[1.0, 10.0, 100.0]}]

In [ ]:
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(random_state=0))
])

In [ ]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid=param_grid,
                          scoring='accuracy', cv=3,
                          verbose=1)

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

## Working with bigger data

In [ ]:
import numpy as np

In [ ]:
import re

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('english')

In [ ]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [ ]:
def stream_docs(path):
    with open(path, 'r', encoding='utf8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path='./movie_data.csv'))

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [ ]:
vect = HashingVectorizer(decode_error='ignore',
                        n_features =2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)

In [ ]:
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

In [ ]:
doc_stream = stream_docs(path='./movie_data.csv')

In [ ]:
import pyprind

In [ ]:
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])

In [ ]:
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [ ]:
X_test, y_test = get_minibatch(doc_stream=doc_stream, size=5000)

In [ ]:
X_test = vect.transform(X_test)

In [ ]:
print('Accuracy: {:.3f}'.format(clf.score(X_test, y_test)))

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
print('Accuracy: {:.3f}'.format(clf.score(X_test, y_test)))

In [ ]:
import pickle
import os

In [ ]:
dest = os.path.join('movieclassifier', 'pkl_objects')

In [ ]:
if not os.path.exists(dest):
    os.makedirs(dest)

In [ ]:
pickle.dump(stop,
           open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
           protocol=4)

In [ ]:
pickle.dump(clf, 
           open(os.path.join(dest, 'classifier.pkl'), 'wb'),
               protocol=4)